In [1]:
import torch
from transformers import pipeline
from datasets import load_dataset

device = "cuda:6" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model="collabora/whisper-medium-hindi",
  chunk_length_s=30,
  device=device,
)


Device set to use cuda:6
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [26]:
ds = load_dataset("Sparsh7/Sanyam_Hindi", split="train")
# sample = ds[2]["audio"]


Resolving data files:   0%|          | 0/5784 [00:00<?, ?it/s]

wavs/M2_chunk04439.wav:   0%|          | 0.00/294k [00:00<?, ?B/s]

wavs/M2_chunk04443.wav:   0%|          | 0.00/365k [00:00<?, ?B/s]

wavs/M2_chunk04442.wav:   0%|          | 0.00/154k [00:00<?, ?B/s]

wavs/M2_chunk04446.wav:   0%|          | 0.00/64.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5783 [00:00<?, ? examples/s]

In [34]:
ds['audio']

[{'path': '/Data/deepakkumar/.cache/huggingface/hub/datasets--Sparsh7--Sanyam_Hindi/snapshots/e2da40430b0f095e8dce83a7f0bb6fd2c092218f/wavs/M1_chunk00004.wav',
  'array': array([-1.22070312e-04, -1.22070312e-04, -6.10351562e-05, ...,
         -3.66210938e-03, -3.66210938e-03, -3.60107422e-03], shape=(120000,)),
  'sampling_rate': 16000},
 {'path': '/Data/deepakkumar/.cache/huggingface/hub/datasets--Sparsh7--Sanyam_Hindi/snapshots/e2da40430b0f095e8dce83a7f0bb6fd2c092218f/wavs/M1_chunk00005.wav',
  'array': array([-0.00189209, -0.00234985, -0.00219727, ..., -0.00064087,
         -0.00082397, -0.00076294], shape=(62400,)),
  'sampling_rate': 16000},
 {'path': '/Data/deepakkumar/.cache/huggingface/hub/datasets--Sparsh7--Sanyam_Hindi/snapshots/e2da40430b0f095e8dce83a7f0bb6fd2c092218f/wavs/M1_chunk00006.wav',
  'array': array([ 6.10351562e-05, -1.83105469e-04, -2.44140625e-04, ...,
          1.16882324e-02,  1.14135742e-02,  1.14746094e-02], shape=(163200,)),
  'sampling_rate': 16000},
 {'pa

In [45]:
import yt_dlp

def download_audio(url, output_path="downloads"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',   # can be "wav", "m4a" etc.
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Example usage
download_audio("https://youtu.be/I6i58To5qxU?si=6IZy0ghzHqZlEBgK")


[youtube] Extracting URL: https://youtu.be/I6i58To5qxU?si=6IZy0ghzHqZlEBgK
[youtube] I6i58To5qxU: Downloading webpage
[youtube] I6i58To5qxU: Downloading tv simply player API JSON
[youtube] I6i58To5qxU: Downloading tv client config
[youtube] I6i58To5qxU: Downloading tv player API JSON
[info] I6i58To5qxU: Downloading 1 format(s): 251
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: downloads/Shahrukh Khan Best Acting - Devdas Movie ｜ Shah Rukh Khan Dialogues ｜ Devdas Movie Scenes.webm
[download] 100% of   13.57MiB in 00:00:02 at 4.53MiB/s     
[ExtractAudio] Destination: downloads/Shahrukh Khan Best Acting - Devdas Movie ｜ Shah Rukh Khan Dialogues ｜ Devdas Movie Scenes.mp3
Deleting original file downloads/Shahrukh Khan Best Acting - Devdas Movie ｜ Shah Rukh Khan Dialogues ｜ Devdas Movie Scenes.webm (pass -k to keep)


In [36]:
arr=ds['audio'][2]['array']

In [37]:
import soundfile as sf

sf.write("out.wav",arr, 16000)  # 16kHz sample rate


In [ ]:
# ds[2]["transcription"]

'Last Monday was perfect—I got the job I’d been dreaming of! I screamed, hugged my parents, and we even had cake for breakfast. But two days later, my mom called—our dog Bruno had passed away in his sleep. I didn’t even get to say goodbye. And just when I thought the week couldn’t get any crazier, I got a surprise call—I was selected for a global fellowship I never even applied for! But earlier that day, my Wi-Fi cut out again during an important meeting... third time this week! It’s beyond frustrating. What a week. Honestly, I’m still processing it all.'

In [46]:
# prediction = pipe(sample.copy(), batch_size=8)["text"]

# we can also return timestamps for the predictions
# prediction = pipe(sample.copy(), batch_size=2, return_timestamps=True)["chunks"]

prediction = pipe("/Data/deepakkumar/Subham_work/downloads/srk.mp3",return_timestamps=True)

In [47]:
prediction

{'text': " देव बाबू क्या हुआ देव बाबू देव बाबू ऐसे भी क्या बेरुखी है देव बाबू कि हम आपको छू भी नहीं सकते अगर यही बात थी तो क्यों आये हमारी जिंदगी में हम भी खुश थे अपनी तन्हाई में क्यों मसीहा बनकर हमारे दर्द को और बढ़ा रहे हो क्यों चंद्रमुखी देखा चली गई ना खांसी अगर ध्यान बदल दो तो गहरे से गहरा जख्म भी भर सकता है देवबाबू वही तो कर रहे हैं हम बस भी करो बहुत पी चुके यही कह दिया होता बहुत जी चुकी कैसे कह दूँ तुम्हारे पास तो पारो है पारो की यादें हैं मेरे पास तो सिर्फ तुम हो पारसमणि जिसे छू के लोहा भी सोना बन जाता है अपनी कीमत बढ़ा रही हो या मेरी रिश्तों के दुनिया में तबाही की कीमत ही क्या होती है कुछ भी नहीं और वैसे भी आपने तो हमें छूने का अधिकार ही नहीं दिया इश्क कर दिया वो सिर्फ यही पूछ लिया होता सांसे लेती हो चंद्रमुखी सांसे लेती हो चंद्रमुखी सांसे लेती हो चंद्रमुखी क्या पाओगी न मेरे पास घर है न दिल सिर्फ पानी का नाम ही तो प्यार नहीं है न देव बाबू प्यार का कारोबार तो बहुत बार किया हैं मगर प्यार सिर्फ एक बार चंद्रमुखी इस भरे पैमाने में और शराब डालो की तो क्या होगा छलकर जमीन पर आकर रहेगी 

In [5]:
from pydub import AudioSegment

# Load audio
audio = AudioSegment.from_file("/Data/deepakkumar/Subham_work/downloads/srk.mp3")

# Define segment (in milliseconds)
start_time = 31 * 1000  # 30 sec
end_time = 45 * 1000    # 50 sec

# Extract segment
segment = audio[start_time:end_time]

# Save it
segment.export("/Data/deepakkumar/Subham_work/downloads/sad.wav", format="wav")


<_io.BufferedRandom name='/Data/deepakkumar/Subham_work/downloads/sad.wav'>

In [2]:
!pip install pydub

In [ ]:
!pip install spleeter


  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of exceptiongroup to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 9.9 MB/s  0:00:546m0:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.8 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 10.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s  0:00:00
